In [131]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

load_dotenv()
model = genai.GenerativeModel(model_name='gemini-1.5-flash',
                              system_instruction="""
Conduct **Relationship Extraction** on the given input. 

### **Expected Output Format**
Respond **ONLY** with two Python lists, separated by `@`:
1. **Entities List**: List of tuples containing (entity_name, classification), where:
   - `entity_name`: Maximum of **three words**.
   - `classification`: Must be one of the following:
     - `PER` (Person)
     - `ORG` (Organisation)
     - `LOC` (Location)
     - `EVT` (Event)
   - **No duplicate entities allowed.**
   - **Every entity must have at least one relationship.**

2. **Relationships List**: List of tuples containing (entity_1, entity_2, relationship_label, relationship_classification), where:
   - `entity_1`, `entity_2`: Must be present in the **Entities List**.
   - `relationship_label`: Maximum of **three words**.
   - `relationship_classification`: Must be one of the following:
     - `PV` (Positive sentiment)
     - `NG` (Negative sentiment)
     - `NE` (Neutral sentiment)
   - **No duplicate relationships allowed.**
   - **Ensure that every identified relationship is valid and meaningful.**
   - **All relationships must have entities that exist in the Entities List.**

### **Processing Rules**
- **Strictly use the specified classifications**; do not identify anything outside them.
- **Ensure high accuracy** in entity recognition and classification.
- **Each entity must be related to at least one other entity**.
- **Ensure that every relationship label is concise, meaningful, and accurate**.
- **Avoid misclassifying generic nouns as entities unless they fit within the specified categories.**

### **Example Output Format**
[("John Doe", "PER"), ("Google", "ORG"), ("New York", "LOC"), ("Tech Summit", "EVT")] @ 
[("John Doe", "Google", "works at", "PV"), ("Tech Summit", "Google", "sponsored by", "PV")]
""")
genai.configure(api_key=os.getenv("GEMINI_KEY"))

In [2]:
def prompt(input):
    response = model.generate_content(input)
    return response.text

In [4]:
def str_to_ls(str):
    entities_str, relationships_str = str.split('@')
    entities = eval(entities_str.strip())
    relationships = eval(relationships_str.strip())
    return entities, relationships


In [ ]:
test = df_grouped['Text'][0]
res = prompt(test)
print(res)
entities, relationships = str_to_ls(res)

for ent1, ent2, rs, classification in relationships:
    if ent1 not in [entity for entity, _ in entities]:
        entities.append((ent1, "MISC"))
    if ent2 not in [entity for entity, _ in entities]:
        entities.append((ent2, "MISC"))


[('Vendor 1', 'ORG'), ('Vendor 2', 'ORG'), ('Pristina Airport', 'ORG'), ('Vendor 3', 'ORG'), ('Vendor 4', 'ORG'), ('Kosovo', 'LOC'), ('ITF', 'ORG'), ('Kosovo Organised Crime Bureau', 'ORG'), ('International Prosecutor', 'PER'), ('UNMIK Department of Justice', 'ORG')] @
[('Vendor 1', 'Vendor 2', 'same owner', 'NE'), ('Vendor 1', 'Pristina Airport', 'tendered', 'NE'), ('Vendor 2', 'Pristina Airport', 'tendered', 'NE'), ('Vendor 1', 'Kosovo', 'based in', 'NE'), ('Vendor 2', 'Kosovo', 'based in', 'NE'), ('Vendor 1', 'Vendor 2', 'competed with', 'NG'), ('Vendor 3', 'Pristina Airport', 'tendered', 'NE'), ('Vendor 4', 'Pristina Airport', 'tendered', 'NE'), ('Vendor 3', 'Prizren', 'located in', 'NE'), ('ITF', 'Kosovo Organised Crime Bureau', 'supported by', 'NE'), ('ITF', 'International Prosecutor', 'submitted to', 'NE'), ('UNMIK Department of Justice', 'International Prosecutor', 'part of', 'NE')]



In [217]:
from pyvis.network import Network

# Create PyVis Network
net = Network(width="100vh", height="100vh", notebook=True, directed=True, cdn_resources='remote', bgcolor='#222222', font_color='white')

# Define dynamic color scheme
node_colors = {
    "PER": "#FFA500",  # Orange
    "ORG": "#007BFF",  # Blue
    "LOC": "#8E44AD",  # Purple
    "EVT": "#1ABC9C",  # Teal
    "MISC": "#BDC3C7"  # Grey
}

relationship_colors = {
    "PV": "#27AE60",  # Green (Positive)
    "NG": "#E74C3C",  # Red (Negative)
    "NE": "#D3D3D3"   # Light Grey (Neutral)
}

# Add nodes with color-coded styles
for entity, classification in entities:
    color = node_colors.get(classification, "#BDC3C7") 
    net.add_node(entity, label=entity, size=30, borderWidth=4, borderWidthSelected=8,
                 color={"highlight": {"border": color}, "background": color, "border": color},
                 font={'size': 18, 'color': 'white'})

# Add edges with relationship colors
for source, target, rs, classification in relationships:
    net.add_edge(source, target, label=rs, width=5,
                 font={'size': 14, 'align': 'middle', 'color': 'white', "strokeColor": "rgba(0,0,0,0)", "vadjust": -10},
                 color=relationship_colors.get(classification, "#FFFFFF"))

# Improve physics settings for better visualization
net.set_options("""
var options = {
    "physics": {
        "barnesHut": {
            "gravitationalConstant": -5050,
            "centralGravity": 0.75,
            "springLength": 180,
            "damping": 0.5,
            "avoidOverlap": 1
        }
    }
}
""")

# Show the network
net.show("test.html")

# Inject **DYNAMIC LEGEND** into HTML file
legend_html = f"""
    <style>
        .legend {{
            position: absolute;
            top: 20px;
            right: 20px;
            background: white;
            padding: 10px;
            border-radius: 5px;
            font-size: 14px;
            font-family: Arial, sans-serif;
            z-index: 1000;
        }}
        .legend-item {{
            display: flex;
            align-items: center;
            margin-bottom: 5px;
        }}
        .legend-color {{
            width: 15px;
            height: 15px;
            margin-right: 5px;
            border-radius: 3px;
        }}
    </style>
    <div class="legend">
        <strong>Legend</strong>
        <div class="legend-item"><div class="legend-color" style="background:{node_colors["PER"]}"></div>Person (PER)</div>
        <div class="legend-item"><div class="legend-color" style="background:{node_colors["ORG"]}"></div>Organisation (ORG)</div>
        <div class="legend-item"><div class="legend-color" style="background:{node_colors["LOC"]}"></div>Location (LOC)</div>
        <div class="legend-item"><div class="legend-color" style="background:{node_colors["EVT"]}"></div>Event (EVT)</div>
        <div class="legend-item"><div class="legend-color" style="background:{node_colors["MISC"]}"></div>Miscellaneous (MISC)</div>
        <hr>
        <div class="legend-item"><div class="legend-color" style="background:{relationship_colors["PV"]}"></div>Positive (PV)</div>
        <div class="legend-item"><div class="legend-color" style="background:{relationship_colors["NG"]}"></div>Negative (NG)</div>
        <div class="legend-item"><div class="legend-color" style="background:{relationship_colors["NE"]}"></div>Neutral (NE)</div>
    </div>
"""

# Append dynamic legend to HTML file
with open("test.html", "a") as f:
    f.write(legend_html)

test.html
